# 데이터 전처리 과정

## 1. 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

## 2. pandas로 csv file 불러오기

In [2]:
train = pd.read_csv('train.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])

## 3. train data를 input과 target으로 나눠주기

In [3]:
X = train.drop('임신 성공 여부', axis=1) # input
y = train['임신 성공 여부'] # target

## 4. 범주형 열 리스트 선언

In [4]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

## 5. 범주형 열의 모든 data를 문자열로 변환

In [5]:
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

## 6. 범주 data를 정수로 변환하기

In [6]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
# test encoding 시 기존에 없던 범주를 발견하면 -1로 처리

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])
# test는 train data를 기준으로 변환하기에 같은 ordinal_encoder 객체 사용

## 7. 정수형 열 리스트 선언

In [7]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

## 8. 결측치(Nan) 처리

In [8]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

* * *

### 유의할 점

1. 결측치를 단순히 0으로 처리하였음, 이 부분에 대한 개선 가능
2. feature selection 필요. 즉, 쓸모없는 feature를 거르거나 feature간 상관관계 파악 필요

* * *

# 학습 및 테스트 과정

## 1. 학습 및 테스트 코드

In [9]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

pred_proba = model.predict_proba(X_test_encoded)[:, 1]
# predict_proba 사용하는 것에 유의!

sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba

sample_submission.to_csv('./baseline_submit.csv', index=False)

## 2. 결과

![Alt text](./baseline_result.png)

* * *

### 유의할 점
1. 학습모델은 사이킷런의 ExtraTreeClassifier
2. 다른 효과적인 모델을 찾아서 성능 개선 가능 (라이브러리에 대한 공부 필수)
* 이전 기수 블로그 참고 시 주로 트리, 앙상블 모델 사용